<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Send connections to a Google Sheets spreadsheet
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Send+connections+to+a+Google+Sheets+spreadsheet:+Error+short+description">Bug report</a>

**Tags:** #linkedin #network #connections #naas_drivers #csv #automation #content #googlesheets

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-10-16 (Created: 2023-10-16)

**Description:** This notebook will show how to send LinkedIn connections to a Google Sheets spreadsheet. It is usefull for organizations to keep track of their contacts and build relationships.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin, gsheet
import naas
import pandas as pd

### Setup variables
**Mandatory**

[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: Cookie used to authenticate Members and API clients
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation
- `spreadsheet_url`: Google Sheet Spreadsheet URL. Please make sure you shared it with our service account to connect : naas-share@naas-gsheets.iam.gserviceaccount.com
- `sheet_name`: Sheet name. Please create it before sending data into it

**Optional**

- `limit`: This refers to the quantity of connections you wish to retrieve. If you intend to fetch all connections, use -1. Please note that retrieving all connections might be time-consuming.


In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1wMEQdG57eKhpSIYdtK9XVmTMbYD6ovIAy2V9XQanom8/edit#gid=579782200"
sheet_name = "New Sheet"

# Optional
limit = 100

## Model

### Get data from Google Sheet spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
df_gsheet

### Get new connections

In [ ]:
def get_new_connections(
    li_at,
    JSESSIONID,
    limit,
    df_gsheet,
    key="PROFILE_ID"
):
    profiles = []
    if len(df_gsheet) > 0:
        profiles = df_gsheet[key].unique()
    else:
        df = linkedin.connect(li_at, JSESSIONID).network.get_connections(limit=limit)
        return df

    # Get new
    df_new = pd.DataFrame()
    update = True
    while update:
        start = 0
        df = linkedin.connect(li_at, JSESSIONID).network.get_connections(
            start=start, count=100, limit=limit
        )
        new_profiles = df[key].unique()
        for i, p in enumerate(new_profiles):
            if p in profiles:
                update = False
                df = df[:i]
                break
        start += 100
        df_new = pd.concat([df_new, df])
    return df_new

df_new = get_new_connections(
    li_at,
    JSESSIONID,
    limit,
    df_gsheet,
)
print("New connections:", len(df_new))
df_new.head(1)

## Output

### Send to Google Sheet

In [ ]:
gsheet.connect(spreadsheet_url).send(df_new, sheet_name=sheet_name, append=True)